# Catboost hyperparameter tuning

To find better hyperparameters than the default, we can do grid search / some search over 'good' combination of parameters, and select the best as with respect to some metric.

We do this with [Weights&Biases](https://wandb.a). If you need access to that, let me know.

However, this work has been a bit useless.

I rushed into hyperparameter tuning toooo early, and should have tried some feature elimination first.

The final model does not use the best parameters by this notebook (although they are inspired on this)<br>
**One should repeat the process of this notebook for the final model**.

## Import standard libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import time
from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer

from catboost import CatBoostRegressor

## Import custom scripts

In [2]:
import sys, os
sys.path.append(os.getcwd()+ "/../")
from src.data_preprocessing import DataPreprocessing

## Load all the features
The datapreprocessing pipeline is doing quite some stuff, and in a non-efficient manner (I don't have much time for optimizing that :( )
But it should be less than 2 min

In [3]:
dp = DataPreprocessing(df_path = "../data/real_estate_ads_2022_10.csv",
                        train_indices_path="../data/train_indices.npy", 
                        test_indices_path="../data/test_indices.npy",
                        get_params_from_params=True,
                        get_tfidf_embeddings_flag=True,
                        get_bert_embeddings_flag=True,
                        get_textual_features_flag=True,
                        transform_time_features_flag=True,
                        transform_cyclic_features_flag=True)

## Load the metrics class
That is convenient to compute multiple metrics:
- **explained_variance_score**: Measures the proportion of the variance in the dependent variable that is predictable from the independent variable(s). A score of 1 indicates perfect prediction, while a score of 0 indicates that the model does not explain any of the variance.

- **r2_score**: Also known as the coefficient of determination, it indicates the proportion of the variance in the dependent variable that is predictable from the independent variable(s). A value of 1 indicates a perfect fit, while a value of 0 indicates that the model does not explain any of the variance.

- **mean_absolute_percentage_error (MAPE)**: Measures the average of the absolute percentage errors of predictions. It provides a percentage error which is easy to interpret but can be sensitive to very small actual values.

- **median_absolute_error**: Computes the median of all absolute differences between the target and predicted values. This metric is robust to outliers and gives a better sense of the typical error when outliers are present.

- **mean_squared_error (MSE)**: Measures the average of the squares of the errors—that is, the average squared difference between the estimated values and the actual value. It penalizes larger errors more than smaller ones due to squaring.

- **mean_squared_log_error (MSLE)**: Similar to MSE but takes the logarithm of the predictions and actual values. It is useful when you want to penalize underestimation more than overestimation and is less sensitive to large errors than MSE.

- **custom metrics**: Compute the percentage of times that the error falls less than some threshold. This may correlate with customer satisfaction, if they are for example happy if there's less than a 5% rate, this would count the percentage of happy customers. Of course, this will need further study (for example, segmenting the score)

In [4]:
import importlib
import src.compute_metrics
importlib.reload(src.compute_metrics) # We do this for debugging purposes

from src.compute_metrics import Metrics

## Split train / test data
We can use the datapreprocessing method for that.

This is done for better reproducibility, but can be done with the sklearn train / test split, and setting a seed should suffice.

In [5]:
X_train, X_test = dp.get_train_test_split(dp.X)
y_train, y_test = dp.get_train_test_split(dp.Y)

## Define convenience functions for prettier display

In [6]:
def filter_metrics(metrics_dict, only_validation=True, format_mean_std_together=True):

    if only_validation:
        metrics_dict = {key: value for key, value in metrics_dict.items() if "test_" in key}

    if format_mean_std_together:
        metrics_dict = {key: f"{value:.2f} ± {metrics_dict[key+'_std']:.2f}" for key, value in metrics_dict.items() if "std" not in key}

    return metrics_dict

def highlight_max(s):
    is_max = s == s.replace("nan ± nan", "0").apply(lambda x: x.split("+-")[0]).max()
    return ['font-weight: bold' if v else '' for v in is_max]

def highlight_min(s):
    is_min = s == s.replace("nan ± nan", "0").apply(lambda x: x.split("+-")[0]).min()
    return ['font-weight: bold' if v else '' for v in is_min]

def format_results_df(results, column_names=None):
    results_df = pd.DataFrame(results).T

    if column_names is not None:
        results_df.columns = column_names
    
    def apply_highlight(column):
        if column.name in ["test_explained_variance", "test_r2", "test_custom_1", "test_custom_5", "test_custom_10", "test_custom_20"]:
            return highlight_max(column)
        else:
            return highlight_min(column)

    
    return results_df.style.apply(apply_highlight, axis=1)

## Some hyperparameter tuning

We can use Weights&Biases.

In [7]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sergi-andreu. Use `wandb login --relogin` to force relogin


True

In [8]:
X = X_train.copy()
y = y_train.copy()

def wandb_train(config=None):

    with wandb.init(config=config):
        config = wandb.config

        bst = CatBoostRegressor(**config)

        metrics = Metrics(dp=dp, backward_transform_flag=True, backward_standardize_flag=False)

        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for train_index, test_index in kf.split(X, y):
            X_train, X_val = X.iloc[train_index], X.iloc[test_index]
            y_train, y_val = y.iloc[train_index], y.iloc[test_index]

            bst.fit(X_train, y_train)
            y_pred = bst.predict(X_val)
                
            computed_metrics = metrics.get_single_train_val_metrics(bst, X_train, y_train, X_val, y_val)
            metrics.append(computed_metrics)

        average_metrics = metrics.get_average()
        std_metrics = metrics.get_std()
        # Add _std to the keys to differentiate them from the average metrics:
        std_metrics = {f"{key}_std" : value for key, value in std_metrics.items()} 

        wandb.log({**average_metrics, **std_metrics})

    return {**average_metrics, **std_metrics}

## Define a sweep

In [9]:
# Define a method for hyperparameter optimization
# It can be grid search, random, or bayesian search

# We use grid search. The reason being that, although it is more computationally expensive,
# it is very effective, and would not create noise / overfitting in the hyperparameter search

# It is best for a first exploration

sweep_config = {
    'method' : 'bayes'
}

# The metrics would not be used (since using grid search)
# Just adding in case we want to use another method later

metric = {
    'name' : 'test_custom_5',
    'goal' : 'maximize'
}

sweep_config['metric'] = metric

"""
# Now the important part: the parameters to sweep, for a catboost classifier:
parameters_dict = {
    'iterations': { # Number of iterations
        'values' : [1000, 5000] # [100, 200, 500]
    },
    'learning_rate' : { # Learning rate
        'values' : [0.005, 0.01, 0.05, 0.1, 0.022760000079870224] # [0.001, 0.01, 0.1]
    },
    'depth' : { # Depth of the tree
        'values' :  [4, 6, 8] # [1, 5, 10]
    },
    'subsample' : { # Subsample ratio
        'values' : [0.800000011920929, 1] #[0.05, 0.5, 1]
    },
    'colsample_bylevel' : { # Column subsample ratio
        'values' : [1] #[0.05, 0.5, 1]
    },
    'min_data_in_leaf' : { # Minimum number of data in leaf
        'values' : [1, 20, 50] #[1, 5, 20, 50, 100]
    },
}
"""

parameters_dict = {
    'iterations': { # Number of iterations
        'values' : [1000, 2000]
    },
    'learning_rate' : { # Learning rate
        'distribution' : 'log_uniform_values',
        'min' : 0.005,
        'max' : 0.1
    },
    'depth' : { # Depth of the# [1, 5, 10]
        'values' :  [2, 5, 6, 10] 
    },
    'subsample' : { # Subsample ratio
        'distribution' : 'uniform',
        'min' : 0.8,
        'max' : 1
    },
    'colsample_bylevel' : { # Column subsample ratio
        'values' : [0.5, 1] #[0.05, 0.5, 1]
    },
    'min_data_in_leaf' : { # Minimum number of data in leaf
        'values' : [1, 20] #[1, 5, 20, 50, 100]
    },
}

parameters_dict = {
    'iterations': { # Number of iterations
        'values' : [5000]
    },
    'learning_rate' : { # Learning rate
        'distribution' : 'log_uniform_values',
        'min' : 0.005,
        'max' : 0.1
    },
    'depth' : { # Depth of the# [1, 5, 10]
        'values' :  [10, 15] 
    },
    'subsample' : { # Subsample ratio
        'distribution' : 'uniform',
        'min' : 0.9,
        'max' : 1
    },
    'colsample_bylevel' : { # Column subsample ratio
        'values' : [1] #[0.05, 0.5, 1]
    },
    'min_data_in_leaf' : { # Minimum number of data in leaf
        'values' : [1] #[1, 5, 20, 50, 100]
    },
}

sweep_config["parameters"] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="olx")

sweep_config

Create sweep with ID: pvw4pbc7
Sweep URL: https://wandb.ai/sergi-andreu/olx/sweeps/pvw4pbc7


{'method': 'bayes',
 'metric': {'name': 'test_custom_5', 'goal': 'maximize'},
 'parameters': {'iterations': {'values': [5000]},
  'learning_rate': {'distribution': 'log_uniform_values',
   'min': 0.005,
   'max': 0.1},
  'depth': {'values': [10, 15]},
  'subsample': {'distribution': 'uniform', 'min': 0.9, 'max': 1},
  'colsample_bylevel': {'values': [1]},
  'min_data_in_leaf': {'values': [1]}}}

In [10]:
%%capture
wandb.agent(sweep_id, function=wandb_train);